In [2]:
import pandas as pd

In [3]:
import datetime

In [4]:
import requests

In [5]:
def historical_fetch(api_key,year,month):
    url = 'https://api.nytimes.com/svc/archive/v1/'+year+'/'+month+'.json'
    article = []
    response = requests.get(url,params = {"api-key":api_key})
    if response.status_code == 200:
        data = response.json()
        print("Success")
        article.extend(data["response"]["docs"])
    else:
        print(response.status_code)    
    return article

In [6]:
api_key = "e9fu5FKIXZHEPv6YFwFdnCa8Zxc3AcHY"

In [7]:
from itertools import product
import time

year = ['2023','2024']
month = ['1','2','3','4','5','6','7','8','9','10','11','12']
year_month = list(product(year, month))

df = historical_fetch(api_key,'2022','12')
historical_data = pd.DataFrame(df)

for year,month in year_month:

    current_data = historical_fetch(api_key,year,month)
    current_df = pd.DataFrame(current_data)
    historical_data = pd.concat([historical_data, current_df], ignore_index=True)

    if year == '2024' and month == '4':
        break

    time.sleep(12)


Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success


ConnectionError: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))

In [14]:
historical_data.shape

(65363, 20)

In [16]:
import os

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "C:/Users/bharg/Desktop/DATA 225/Group Project 9/crack-will-422608-j1-166ab9162e68.json"

In [17]:
from google.cloud import bigquery

from google.api_core.exceptions import NotFound

# Initialize a BigQuery client
client = bigquery.Client()

In [18]:
schema = [
    bigquery.SchemaField("abstract", "STRING"),
    bigquery.SchemaField("web_url", "STRING"),
    bigquery.SchemaField("snippet", "STRING"),
    bigquery.SchemaField("lead_paragraph", "STRING"),
    bigquery.SchemaField("print_section", "STRING"),
    bigquery.SchemaField("print_page", "STRING"),
    bigquery.SchemaField("source", "STRING"),
    bigquery.SchemaField("multimedia", "RECORD", mode="REPEATED", fields=[
        bigquery.SchemaField("rank", "INT64"),
        bigquery.SchemaField("subtype", "STRING"),
        bigquery.SchemaField("caption", "STRING"),
        bigquery.SchemaField("credit", "STRING"),
        bigquery.SchemaField("type", "STRING"),
        bigquery.SchemaField("url", "STRING"),
        bigquery.SchemaField("height", "INT64"),
        bigquery.SchemaField("width", "INT64"),
        bigquery.SchemaField("crop_name", "STRING"),
        bigquery.SchemaField("legacy", "RECORD", fields=[
            bigquery.SchemaField("xlarge", "STRING"),
            bigquery.SchemaField("xlargewidth", "INT64"),
            bigquery.SchemaField("xlargeheight", "INT64"),
            bigquery.SchemaField("thumbnail", "STRING"),
            bigquery.SchemaField("thumbnailwidth", "INT64"),
            bigquery.SchemaField("thumbnailheight", "INT64"),
            bigquery.SchemaField("wide", "STRING"),
            bigquery.SchemaField("widewidth", "INT64"),
            bigquery.SchemaField("wideheight", "INT64"),
        ]),
    ]),
    bigquery.SchemaField("headline", "RECORD", fields=[
        bigquery.SchemaField("main", "STRING"),
        bigquery.SchemaField("kicker", "STRING"),
        bigquery.SchemaField("content_kicker", "STRING"),
        bigquery.SchemaField("print_headline", "STRING"),
        bigquery.SchemaField("name", "STRING"),
        bigquery.SchemaField("seo", "STRING"),
        bigquery.SchemaField("sub", "STRING"),
    ]),
    bigquery.SchemaField("keywords", "RECORD", mode="REPEATED", fields=[
        bigquery.SchemaField("name", "STRING"),
        bigquery.SchemaField("value", "STRING"),
        bigquery.SchemaField("rank", "INT64"),
        bigquery.SchemaField("major", "STRING"),
    ]),
    bigquery.SchemaField("pub_date", "TIMESTAMP"),
    bigquery.SchemaField("document_type", "STRING"),
    bigquery.SchemaField("news_desk", "STRING"),
    bigquery.SchemaField("section_name", "STRING"),
    bigquery.SchemaField("byline", "RECORD", fields=[
        bigquery.SchemaField("original", "STRING"),
        bigquery.SchemaField("person", "RECORD", mode="REPEATED", fields=[
            bigquery.SchemaField("firstname","STRING"),
            bigquery.SchemaField("middlename","STRING"),
            bigquery.SchemaField("lastname","STRING"),
            bigquery.SchemaField("qualifier","STRING"),
            bigquery.SchemaField("title","STRING"),
            bigquery.SchemaField("role","STRING"),
            bigquery.SchemaField("organization","STRING"),
            bigquery.SchemaField("rank","INT64"),
        ]),
        bigquery.SchemaField("organization", "STRING"),
    ]),
    bigquery.SchemaField("type_of_material", "STRING"),
    bigquery.SchemaField("_id", "STRING"),
    bigquery.SchemaField("word_count", "INT64"),
    bigquery.SchemaField("uri", "STRING"),
    bigquery.SchemaField("subsection_name", "STRING"),
    bigquery.SchemaField("run_timestamp", "TIMESTAMP"),
]

In [19]:
table_id = "crack-will-422608-j1.GroupProject.article"

try:
    client.get_table(table_id)
    # If the table exists, delete it
    client.delete_table(table_id)
    print(f"Table '{table_id}' already exists. Deleting it...")
except:
    # If the table doesn't exist, proceed to create it
    print(f"Table '{table_id}' doesn't exist. Proceeding to create it...")

table = bigquery.Table(table_id, schema=schema)
table = client.create_table(table)


Table 'crack-will-422608-j1.GroupProject.article' already exists. Deleting it...


In [20]:
import datetime


historical_data['pub_date'] = pd.to_datetime(historical_data['pub_date'])
historical_data['pub_date'] = historical_data['pub_date'].dt.strftime('%Y-%m-%d %H:%M:%S')
historical_data.fillna("", inplace = True)

current_timestamp = datetime.datetime.now()
current_timestamp_str = current_timestamp.strftime('%Y-%m-%d %H:%M:%S')
historical_data['run_timestamp'] = current_timestamp_str

rows_to_insert = historical_data.to_dict(orient="records")
for item in rows_to_insert:
    for multimedia_item in item.get('multimedia', []):
        if 'subType' in multimedia_item:
            del multimedia_item['subType']


In [21]:
len(rows_to_insert)

65363

In [22]:
batch_size = 1000

num_batches = (len(rows_to_insert) + batch_size - 1) // batch_size

for i in range(num_batches):
    start_idx = i * batch_size
    end_idx = min((i + 1) * batch_size, len(rows_to_insert))
    batch_data = rows_to_insert[start_idx:end_idx]

    print(table_id)
    errors = client.insert_rows_json(table_id, batch_data)

    if errors:
        print(f"Errors occurred while uploading batch {i}: {errors}")
    else:
        print(f"Batch {i} uploaded successfully.")

crack-will-422608-j1.GroupProject.article
Batch 0 uploaded successfully.
crack-will-422608-j1.GroupProject.article
Batch 1 uploaded successfully.
crack-will-422608-j1.GroupProject.article
Batch 2 uploaded successfully.
crack-will-422608-j1.GroupProject.article
Batch 3 uploaded successfully.
crack-will-422608-j1.GroupProject.article
Batch 4 uploaded successfully.
crack-will-422608-j1.GroupProject.article
Batch 5 uploaded successfully.
crack-will-422608-j1.GroupProject.article
Batch 6 uploaded successfully.
crack-will-422608-j1.GroupProject.article
Batch 7 uploaded successfully.
crack-will-422608-j1.GroupProject.article
Batch 8 uploaded successfully.
crack-will-422608-j1.GroupProject.article
Batch 9 uploaded successfully.
crack-will-422608-j1.GroupProject.article
Batch 10 uploaded successfully.
crack-will-422608-j1.GroupProject.article
Batch 11 uploaded successfully.
crack-will-422608-j1.GroupProject.article
Batch 12 uploaded successfully.
crack-will-422608-j1.GroupProject.article
Batch 